# Below Code Block contains all the necessary constants fetched from the .env file

### Import Environment Variables

In [54]:
import os
BLOBACCOUNTNAME = os.getenv('BLOBACCOUNTNAME')
BLOBACCESSKEY = os.getenv('BLOBACCESSKEY')
AZUR_ENDPOINT = os.getenv('AZUR_ENDPOINT')
AZUR_OPENAI_KEY = os.getenv('AZUR_OPENAI_KEY')

### get container clinet

In [55]:
# Function to get the container client to store the video in to

from azure.storage.blob import BlobServiceClient
import os
BLOBACCOUNTNAME = os.getenv('BLOBACCOUNTNAME')
BLOBACCESSKEY = os.getenv('BLOBACCESSKEY')
def CreateContainerIfNotExists(container_name):
    """
    Function Information:
        Info : Create container for particular KB for store upload document in this container
    """

    global account_name, account_key

    try:

        blob_service_client = BlobServiceClient(account_url=f"https://{BLOBACCOUNTNAME}.blob.core.windows.net", credential=BLOBACCESSKEY)

        # Azur container client
        container_client = blob_service_client.get_container_client(container_name)
        if not container_client.exists():
            container_client.create_container()
        return True, container_client
    except Exception as e:
        print(e)
        return False, None

### get container SAS token

In [56]:
# Function code to get the Container SAS Token
from datetime import datetime, timedelta
from azure.storage.blob import generate_container_sas, ContainerSasPermissions

def CreateContainerSASToken(container_name='tenant-151-safetycheck'):
    """
    Function information:
        Info : Create SAS token for particular KB for access that document of that KB
    """


    try:
        # SAS token information        
        token_duration = timedelta(hours=1)
        permissions = ContainerSasPermissions(read = True, list=True)
        sas_token = generate_container_sas(
            account_name=BLOBACCOUNTNAME,
            container_name=container_name,
            account_key=BLOBACCESSKEY,
            permission=permissions,
            expiry=  datetime.utcnow() + token_duration
        )
        return sas_token
    except Exception as e:
        print(e)
        return False

### Upload Video to Storage account

In [57]:
# Code to upload Video to AZURE blob storage account
import uuid
def UploadVideoToStorageAccount(file_path, container_name = 'tenant-151-safetycheck'):
    # file_path = '/content/test-video-101.mp4'
    with open(file_path, 'rb') as video_file:
        create_container_status, container_client = CreateContainerIfNotExists(container_name)

        file_extension = file_path.split(".")[-1]
        upload_file_name = f"test-videos/{uuid.uuid4()}.{file_extension}"

        # Upload blob
        blob_client = container_client.get_blob_client(upload_file_name)
        video_data = video_file.read()
        blob_client.upload_blob(data=video_data, overwrite=True)

        # Generate video URL
        video_url = blob_client.url
        print(f"Video successfully uploaded: {video_url}")
        return video_url
    
    print("something went wrong")

### Get Bearer Token

In [58]:
# GET API bearer token:
import requests

def get_api_bearer_token():
    '''
    Function to get bearer token that can be used for getting access token for video indexing
    '''

    # Define the API endpoint
    tenant_id =  os.getenv('AZURE_VIDEO_INDEXER_TENANT_ID')
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

    # Prepare the data and files
    client_id = os.getenv('AZURE_VIDEO_INDEXER_CLIENT_ID')
    client_secret = os.getenv('AZURE_VIDEO_INDEXER_CLIENT_SECRET')
    grant_type = "client_credentials"
    scope = "https://management.azure.com/.default"

    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": grant_type,
        "scope": scope
    }

    # Send the request
    response = requests.post(url, data=data)

    # Handle the response
    if response.status_code == 200:
        return response.json()['access_token']
    else:
        print("Failed to get bearer token")
        print("Status code:", response.status_code)
        print("Response:", response.text)


### get access token

In [59]:
import requests

def get_access_token():
    '''
    Function to get bearer token that can be used for getting access token for video indexing
    '''
    subscription_id = os.getenv('AZURE_VIDEO_INDEXER_SUBSCRIPTION_ID')
    resource_group = os.getenv('AZURE_VIDEO_INDEXER_RESOURCE_GROUP')
    account_name = os.getenv('AZURE_VIDEO_INDEXER_ACCOUNT_NAME')
    version = '2022-08-01'

    url = f"https://management.azure.com/subscriptions/{subscription_id}/resourcegroups/{resource_group}/providers/Microsoft.VideoIndexer/accounts/{account_name}/generateAccessToken?api-version={version}"
    
    data = {
        "permissionType": "Contributor",
        "scope": "Account"
    }

    bearer_token = get_api_bearer_token()

    headers = {
        "Content-Type": 'application/json',
        "Authorization": f"Bearer {bearer_token}"
    }

    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()['accessToken']
    else:
        print("Failed to get access token")
        print("Status code:", response.status_code)
        print("Response:", response.text)

### Functionality for uploading video using video url to Azure Video Indexer 

In [60]:
# Upload video for indexing

########### Python 3.2 #############
import urllib.request, json, requests
from urllib.parse import quote


def upload_video_for_indexing(video_url):

    try:

        access_token = get_access_token()
        account_id = os.getenv('AZURE_VIDEO_INDEXER_ACCOUNT_ID')
        url = f"https://api.videoindexer.ai/eastus/Accounts/{account_id}/Videos?name=test-844&videoUrl={quote(video_url)}&accessToken={access_token}&preventDuplicates=False"
        hdr ={
        # Request headers
        'Ocp-Apim-Subscription-Key':  os.getenv('AZURE_VIDEO_INDEXER_OCM_SUBSCRIPTION_KEY')
        }

        request = urllib.request.Request(url, headers=hdr, method='POST')
        response = urllib.request.urlopen(request)
        
        if response.status == 200:
            print("video uploaded successfully")
            response_data = json.loads(response.read().decode('utf-8'))
            print(response_data)
            return response_data
        else:
            print("Failed to upload video")
            print("Status code:", response.status)
            response_body = response.read().decode('utf-8')
            print("Response:", response_body)
    except urllib.error.HTTPError as e:
        print("HTTP Error:", e.code, e.reason)
    except urllib.error.URLError as e:
        print("URL Error:", e.reason)
    except Exception as e:
        print("Something went wrong")
        print(e)
####################################

### Main functions to ask questions and get answers from openAi 

In [61]:
from openai import OpenAI
def ask_question_on_video(video_id, question,context) -> str:
    message_history = [
        {   'role': "system", 
             "content": 'You are an virtual assistant who will take in context and a question and answer the question only based on question. Provide a concise answer, and if you cannot find answer in the context, just state: "I am not able to answer the question."',
        },
        {
            "role": "user", 
            "content": context
        },
        {
            "role": "user", 
            "content": question
        }
    ]

    client = OpenAI(api_key=os.getenv('OpenAI_KEY'))
    chat_response = client.chat.completions.create(
        model=os.getenv('ASSISTANT_MODEL'), 
        messages=message_history,
        max_tokens=1000, 
        temperature=float(os.getenv('TEMPERATURE')),
    )
    answer = chat_response.choices[0].message.content
    answer = answer if answer != 'I am not able to answer the question.' else ''
    return answer

### Async Function to Run until the video goes from processing state to processed state......

In [62]:
import time
from typing import Optional

def wait_for_index_async(video_id:str) -> None:
    '''
    Calls getVideoIndex API in 10 second intervals until the indexing state is 'processed'
    (https://api-portal.videoindexer.ai/api-details#api=Operations&operation=Get-Video-Index).
    Prints video index when the index is complete, otherwise throws exception.

    :param video_id: The video ID to wait for
    :param language: The language to translate video insights
    '''
    accountId = os.getenv('AZURE_VIDEO_INDEXER_ACCOUNT_ID')
    access_token = get_access_token()
    url = f'https://api.videoindexer.ai/eastus/Accounts/{accountId}/Videos/{video_id}/Index'

    params = {
        'accessToken': access_token
    }

    print(f'Checking if video {video_id} has finished indexing...')
    processing = True
    start_time = time.time()
    while processing:
        response = requests.get(url, params=params)

        response.raise_for_status()

        video_result = response.json()
        video_state = video_result.get('state')

        if video_state == 'Processed':
            processing = False
            print(f'The video index has completed. Here is the full JSON of the index for video ID {video_id}: \n{video_result}')
            video_insights = video_result
            transcripts = video_insights.get('videos')[0].get('insights').get('transcript')
            print("transcripts", transcripts)
            context = ' '.join([transcript['text'] for transcript in transcripts])
            context += ' \n '
            
            while True:
                user_question = input("Please enter your question (or type 'exit' to finish): ")
                if user_question.lower() == 'exit':
                    break
                answer = ask_question_on_video(video_id, user_question, context)
                print(f"Answer: {answer}")
            
        elif video_state == 'Failed':
            processing = False
            print(f"The video index failed for video ID {video_id}.")
            break

        print(f'The video index state is {video_state}')
 
        time.sleep(10)


### Get Video Indexing using video id, recieved in response of Upload video indexing API

In [63]:
# get video index using video id got from uploading video for indexing
from pprint import pprint
import urllib.request
import json

########### Python 3.2 #############
def get_video_indexing(video_id):
    try:
        access_token = get_access_token()
        accountId = os.getenv('AZURE_VIDEO_INDEXER_ACCOUNT_ID')
        url = f"https://api.videoindexer.ai/eastus/Accounts/{accountId}/Videos/{video_id}/Index?includeSummarizedInsights=true&accessToken={access_token}"

        hdr ={
        # Request headers
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': os.getenv('AZURE_VIDEO_INDEXER_OCM_SUBSCRIPTION_KEY'),
        }

        req = urllib.request.Request(url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        response_data = response.read().decode("utf-8")
        response_json = json.loads(response_data)
        return response_json

    except Exception as e:
        print(e)  
####################################

### Main Python Code:

In [64]:
file_path = input("Enter video file path:")

print('Uploading Video to Azure Storage account.')
# default container to store test videos is set as 'tenant-151-safetycheck', can be changed by adding container name in the UploadVideoToStorageAccount as 2nd argument.
video_url = UploadVideoToStorageAccount(file_path)

print('Generating video url with embedded SAS Token.')
# in the below step also in Create ContainerSASToken function default container is set as tenant-151-safetycheck, can be updated by passing the container name as an argument to the CreateContainerSASToken
access_video_url = f'{video_url}?{CreateContainerSASToken()}'

print('Uploading Video to Azure Video Indexer.')
response_data = upload_video_for_indexing(access_video_url)

azure_video_id = response_data['id']
print('Checking Video Processing State.')
wait_for_index_async(azure_video_id)



Checking if video xo2go3wk9v has finished indexing...
The video index has completed. Here is the full JSON of the index for video ID xo2go3wk9v: 
{'partition': None, 'description': None, 'privacyMode': 'Private', 'state': 'Processed', 'accountId': '1d0b01cc-6f4b-48ad-bb1e-a028d29163a6', 'id': 'xo2go3wk9v', 'name': 'test-844', 'userName': ' ', 'created': '2024-12-24T07:32:15.29+00:00', 'isOwned': True, 'isEditable': True, 'isBase': True, 'durationInSeconds': 698, 'duration': '0:11:38.681', 'summarizedInsights': {'name': 'test-844', 'id': 'xo2go3wk9v', 'privacyMode': 'Private', 'duration': {'time': '0:11:38.681', 'seconds': 698.7}, 'thumbnailVideoId': 'xo2go3wk9v', 'thumbnailId': '00000000-0000-0000-0000-000000000000', 'faces': [], 'keywords': [{'isTranscript': True, 'id': 1, 'name': 'totally fine', 'appearances': [{'startTime': '0:05:49.8', 'endTime': '0:06:17.32', 'startSeconds': 349.8, 'endSeconds': 377.3}, {'startTime': '0:06:52.96', 'endTime': '0:07:18.92', 'startSeconds': 413, 'end